In [6]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Beispielhafte Daten
# X = np.random.rand(100, 13)
# y = np.random.rand(100, 5)

# Deine echten Daten hier laden
X = ... # Dein Input-Datensatz (100, 13)
y = ... # Dein Output-Datensatz (100, 5)

In [14]:
data = pd.read_csv("initial_data.csv")

print(data)
#print(data.keys())

x = data[['Engine speed', 'Engine load', 'Railpressure', 'Air supply', 'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature']]
y = data[['NOx', 'PM 1', 'CO2', 'PM 2', 'Pressure cylinder']]

Index(['Engine speed', 'Engine load', 'Railpressure', 'Air supply',
       'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature',
       'NOx', 'PM 1', 'CO2', 'PM 2', 'Pressure cylinder'],
      dtype='object')


In [16]:
# Datenaufteilung in Training und Test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Datenstandardisierung
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

# Umwandlung in PyTorch Tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [20]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(8, 16)
        self.layer2 = nn.Linear(16, 16)
        self.layer3 = nn.Linear(16, 10)
        self.output = nn.Linear(10, 5)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.output(x)
        return x

model = NeuralNet()

In [21]:
# Verlustfunktion und Optimierer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Trainingsparameter
num_epochs = 150
batch_size = 10

# Daten in Batches aufteilen
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [22]:
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        # Forward-Pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward-Pass und Optimierung
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Modell speichern (optional)
torch.save(model.state_dict(), 'model.pth')

Epoch [10/150], Loss: 0.9082
Epoch [20/150], Loss: 0.8704
Epoch [30/150], Loss: 0.2025
Epoch [40/150], Loss: 0.0791
Epoch [50/150], Loss: 0.3182
Epoch [60/150], Loss: 0.1381
Epoch [70/150], Loss: 0.1965
Epoch [80/150], Loss: 0.3453
Epoch [90/150], Loss: 0.0730
Epoch [100/150], Loss: 0.1032
Epoch [110/150], Loss: 0.0966
Epoch [120/150], Loss: 0.0922
Epoch [130/150], Loss: 0.1560
Epoch [140/150], Loss: 0.1594
Epoch [150/150], Loss: 0.1127


In [24]:
model.eval()  # Setzt das Modell in den Evaluationsmodus
with torch.no_grad():  # Keine Gradientenberechnung
    y_pred = model(X_test)
    test_loss = criterion(y_pred, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')
    print(y_pred)

Test Loss: 0.3132
tensor([[-1.1002, -0.9322, -1.0134, -0.7805, -1.0543],
        [-0.4275,  0.7038, -0.0429,  1.6560, -0.4129],
        [-0.8314, -0.1408, -0.5709,  0.3913, -0.7900],
        [-0.8045,  0.2496, -0.5947,  1.4952, -0.8004],
        [-1.1009, -0.6918, -1.0726, -0.0335, -1.0800],
        [-0.6690,  0.6462, -0.3513,  2.0625, -0.6538],
        [ 2.5275, -0.7408,  0.3512, -1.0158,  0.9423],
        [-0.0382,  0.8642,  0.8987,  0.1424,  0.6666],
        [-1.1954, -1.4205, -1.4781, -0.7998, -1.2655],
        [-1.3724, -1.7147, -1.6702, -1.1781, -1.4329],
        [-0.9966, -0.5455, -1.1096,  0.6052, -1.0361],
        [-1.3370, -1.4063, -1.4750, -1.0397, -1.3209],
        [ 0.1123,  0.9088,  0.9820,  0.1119,  0.8003],
        [-0.4888,  0.6195, -0.2350,  1.8731, -0.5082],
        [-1.0249, -0.9254, -0.8964, -0.9284, -0.9786],
        [ 1.3995, -1.1178, -0.5982, -0.4100,  0.1069],
        [-1.0686, -0.8506, -0.9923, -0.6114, -1.0289],
        [-0.9441, -0.6459, -0.7890, -0.4502, -0